In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pprint
import time
import datetime
import re

from selenium import webdriver

def get_soup(url):

    header = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36Name"
    }
    time.sleep(1)
    content = requests.get(url , headers = header).text

    soup = BeautifulSoup(content, "html.parser")
    
    return soup

# import re
# def striphtml(data):
#     clean = re.compile('<.*?>')
#     clean = re.sub(clean, ' ', data)
#     clean = clean.replace(u'\xa0', u' ')
#     return clean

# def splithtml(data):
   
#     newclean=[]
#     clean =  re.split(r'<.*?>', data)
#     clean = [data.replace(u'\xa0', u' ') for data in clean]
#     clean2 = [x for x in clean if x.strip()]
#     return clean2
# option = webdriver.ChromeOptions()
# option.add_arg


# exec_path = "chromedriver"
# browser = webdriver.Chrome(executable_path=exec_path, chrome_options=option)

In [ ]:
def parseResults(results, querymonth, queryyear):
    jobpostings = []

    for result in results:
        
        job_info = result.text.split("\n")
        
        date = datetime.datetime.strptime(job_info[6], "%B %d, %Y")
        month = date.month
        year = date.year
        
        
        dct={}
        #only parse info if date in query range
        if ((month==querymonth)&(year==queryyear)):
            
            dct['JOB TITLE'] = job_info[0]
            dct['LOCATION'] = job_info[2]
            dct['FUNCTION'] = job_info[4]
            dct['DATE'] = job_info[6]

            dct['LANGUAGE'] = job_info[-1]
            job_url = result.find_element_by_tag_name('a').get_attribute('href')
            dct['URL'] = job_url
            jobdetails = parseJobDescription(job_url)
            dct.update(jobdetails)

            jobpostings.append(dct)
    
    #return extracted jobpostings and month/year of last posting
    return jobpostings, month, year

def parseJobDescription(url):
    time.sleep(1)
    
    dct = {}
    
    try:
        soup = get_soup(url)

        dct['job_title'] = soup.find('h1',{'class':'job-title'}).text
        dct['location'] = soup.find('div', {'class':"job-location job-categories"}).text.replace("Locations:","").strip()
        dct['function'] = soup.find('span', {'itemprop':"occupationalCategory"}).find_all('span')[-1].text
        dct['requisition_id'] = soup.find('h5', {'class':"job-description-reqid"}).text.replace("Requisition ID:","").strip()
        dct['job_title'] = soup.find('h1',{'class':'job-title'}).text

        jdElements = soup.find('div',{'class':'jibe-job-description'}).find_all(['p','ul','li'])
        jd = []

        for element in jdElements:
            text = element.text.split()
            text = " ".join([word.strip() for word in text])
            if text:
                jd.append(text)

        dct['job_description'] = jd
    except:
        print("error reading job @ {}".format(url))
    return dct



In [ ]:
currentdate = datetime.datetime.now()
querymonth = currentdate.month - 1
queryyear = currentdate.year

month = querymonth
year = queryyear
jobpostings = []

#next up - april
i=54
browser = webdriver.Chrome()
#continue to get search listings if month = querymonth or later
# browser.manage().timeouts().implicitlyWait()
while ((month>=querymonth) & (year == queryyear)):
    url = "{BASEURL}".format(i)
    
  
    try:
        browser.get(url)
        time.sleep(5)
        results = browser.find_elements_by_class_name('job-result')        
        results_parsed, month, year = parseResults(results, querymonth, queryyear)
        
        jobpostings.extend(results_parsed)
        print("Read {}".format(url))
    except:
        print("Error reading {}".format(url))
    i+=1
    time.sleep(1)
    

In [ ]:
len(jobpostings)
pprint.pprint(jobpostings)

In [ ]:
import json

datetoday = datetime.date.today()
date = datetoday.strftime("%Y-%B-%d %H:%M").split()[0]

filename = '{name_}'+date+'.json'
with open(filename, 'w') as fp:
    json.dump(jobpostings, fp, indent=4)

# with open(filename, 'r') as fp:
#     readdata = json.load(fp)
# # pprint.pprint(readdata)

In [ ]:
filename